In [1]:
# import findspark
# spark_path = '../../2016UCSD/255/spark-1.6.0-bin-hadoop2.6/'
# findspark.init(spark_path)
# from pyspark import SparkContext
# sc = SparkContext(master="local[3]")


In [2]:
from get_features import *
import os, pickle
from sklearn import linear_model, datasets
from lib import *

In [3]:
def train_model(path, domain, model, emotion):
    train_file = 'Data/%s/trainData.json' % (domain)
    test_file = 'Data/%s/testData.json' % (domain)

    pickled_trainX = '%s/%s/%s/%s/trainX.pickle' % (path, domain, model, emotion)
    pickled_trainY = '%s/%s/%s/%s/trainY.pickle' % (path, domain, model, emotion)
    pickled_testX = '%s/%s/%s/%s/testX.pickle' % (path, domain, model, emotion)
    pickled_testY = '%s/%s/%s/%s/testY.pickle' % (path, domain, model, emotion)
    
    if not os.path.exists(pickled_trainX) or not os.path.exists(pickled_trainY):
        trainX, trainY = get_json_data(train_file, emotion)
        pickle.dump(trainX, open(pickled_trainX, 'w'))
        pickle.dump(trainY, open(pickled_trainY, 'w'))
    else:
        trainX = pickle.load(open(pickled_trainX, 'r'))
        trainY = pickle.load(open(pickled_trainY, 'r'))
    if not os.path.exists(pickled_testX) or not os.path.exists(pickled_testY):
        testX, testY = get_json_data(test_file, emotion)
        pickle.dump(testX, open(pickled_testX, 'w'))
        pickle.dump(testY, open(pickled_testY, 'w'))
    else:
        testX = pickle.load(open(pickled_testX, 'r'))
        testY = pickle.load(open(pickled_testY, 'r'))
    return trainX, trainY, testX, testY

In [17]:
path, domain, model = 'pickled' , 'Semeval_2007', 'model1'
emotion = 'sad'
print emotion
trainX, trainY, testX, testY = train_model(path, domain, model, emotion)
c = 10 ** (2)
classifier = model1(trainX, trainY, c = c, ml_model = 'logistic')
#print c, compare(classifier, trainX, trainY)
print evaluate(classifier, testX, testY)

sad
logistic
(987, 166) (987,)
precision  0.695533769063
recall  0.644039976656
0.701244813278


In [36]:
#joy, sad, disgust, anger, surprise, fear
path, domain, model = 'pickled' , 'Semeval_2007', 'model1'
for emotion in ['joy', 'sad', 'disgust', 'anger', 'surprise', 'fear']:
    print '======================='
    print emotion
    trainX, trainY, testX, testY = train_model(path, domain, model, emotion)
    for i in[1]:
        c = 10 ** (i)
        classifier = model1(trainX, trainY, c = c, ml_model = 'logistic')
        print '======'
        print 'c, train score, test score,'
        print c
        #print evaluate(classifier, trainX, trainY)
        print  evaluate(classifier, testX, testY)

joy
logistic
(987, 166) (987,)
c, train score, test score,
10
precision  0.616421568627
recall  0.60817805383
0.614107883817
sad
logistic
(987, 166) (987,)
c, train score, test score,
10
precision  0.702780008029
recall  0.647395681354
0.705394190871
disgust
logistic
(987, 166) (987,)
c, train score, test score,
10
precision  0.630594135802
recall  0.618788601909
0.639004149378
anger
logistic
(987, 166) (987,)
c, train score, test score,
10
precision  0.654225255032
recall  0.655771372877
0.655601659751
surprise
logistic
(987, 166) (987,)
c, train score, test score,
10
precision  0.286307053942
recall  0.5
0.572614107884
fear
logistic
(987, 166) (987,)
c, train score, test score,
10
precision  0.67057591623
recall  0.612236461348
0.618257261411


## polarity with semeval-2017-task-5-subtask-1 data and TextBlob model

In [ ]:
from textblob import TextBlob

In [ ]:
trainFile = 'Data/polarity/trainData'
testFile = 'Data/polarity/testData'
trainData = [[ ''.join(line.split(',')[:-1]), float(line.strip('\n').split(',')[-1]) > 0] for line in open(trainFile, 'r').readlines()]
testData = [[ ''.join(line.split(',')[:-1]), float(line.strip('\n').split(',')[-1]) > 0] for line in open(testFile, 'r').readlines()]


In [ ]:
print len(trainData), len(testData)

In [ ]:
def evaluate(data):
    pred = [TextBlob(d[0]).sentiment.polarity > 0 for d in data]
    Y = [d[1] for d in data]
    return np.sum(np.array(pred) == np.array(Y)) / float(len(data))
print evaluate(trainData), evaluate(testData)